In [1]:
import numpy as np
from matplotlib import pyplot as plt
import py_gauge_mc
import os
from tqdm import tqdm
import scipy.special

# Cosine potential

$-S(\theta) = K (1 - \mbox{cos}(\theta))$

$$\int d\theta e^{-S(\theta)} \mbox{cos}(n \theta) = (-1)^n 2\pi J_n(K)$$

But odd numbers always appear in pairs, no sign problem

$\tilde V_n - \tilde V_0 = - \mbox{log} \left( \frac{J_n(K)}{J_0(K)}\right)$

Proof:

Local moves change 3 plaquettes by $+1$ and 3 by $-1$, consider the generalized case of 6 updates of $\pm 1$.
If the graph starts with $n$ odd numbers on it, the first $\pm 1$ brings that to either $n+1$ or $n-1$, and the second to one of $n$, $n+2$, $n-2$. In similar fashion after 6 updates we have $n + 2k$ total odd numbers in the graph with $k$ an integer from $-3$ to $3$. So if $n$ starts even then $n+2k$ remains even.

In [2]:
def v_tilde(ns, k):
    return - np.log(scipy.special.jv(ns, k) / scipy.special.jv(0, k))

In [ ]:
L = 8
n_exp = 1024

inv_ks = np.linspace(1.1,1.3,60)

cosine_all_energies = []
for inv_k in tqdm(inv_ks,desc='K'):
    vns = v_tilde(np.arange(0,100), 1./inv_k).astype(np.float32)
    vns = np.tile(vns, [n_exp, 1])
    
    graph = py_gauge_mc.GPUGaugeTheory((L,L,L,L),vns)
    graph.run_local_update(1000)
    energies = graph.simulate_and_get_energies(100, steps_per_sample=100, run_global_updates=False, run_parallel_tempering=False)
    cosine_all_energies.append(energies)
del graph
cosine_all_energies = np.array(cosine_all_energies)/(L**4)

K:   0%|                                                                                                                                                                                                                                                                                                                                                                                                         | 0/60 [00:00<?, ?it/s]

In [ ]:
cosine_all_energies.shape

In [ ]:
plt.plot(inv_ks, cosine_all_energies.mean(axis=(1,2)))
plt.grid()
plt.show()

plt.plot(inv_ks, cosine_all_energies.var(axis=(1,2)))
plt.grid()
plt.show()